In [94]:
import requests_cache
import requests
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
from pandas.tools.plotting import scatter_matrix
import numpy as np
import seaborn as sns
from requests_oauthlib import OAuth1Session

In [108]:
requests_cache.install_cache("cache")

def yelp_search(term, location):
    """
    makes a search request
    
    Argument: search term, location
    
    Return: search result items as a list (for 0 items, return an empty list)
    """
    consumer_key = "uFKYftN0fu4pD6guW2cYng"
    consumer_secret = "iUYQ2Hk3-vTQVd0zOFnvloWPivA"
    token = "FZnep4FiFmwrrifuNm6M-_K661Wb3J02"
    token_secret = "CRvBo_fO-r4pE_-p2UrnN_mxBlc"
   
    urlbase = "http://api.yelp.com/v2/search"
    params = ({"term": term})
    params.update({"location": location, "sort": "1", "cll" : "latitude,longitude"})
    session = OAuth1Session(consumer_key, consumer_secret,token,token_secret)
     
    request = session.get(urlbase,params=params)
   
    #Transforms JSON API response into dictionary
    data = request.json()
    session.close()
    data_dict = data["businesses"]
    
    return data_dict


In [109]:
data = yelp_search("food", "San Francisco")
data

[{u'categories': [[u'Vietnamese', u'vietnamese'],
   [u'Sandwiches', u'sandwiches']],
  u'display_phone': u'+1-415-503-1421',
  u'id': u'dinosaurs-san-francisco-4',
  u'image_url': u'https://s3-media3.fl.yelpcdn.com/bphoto/oX333wuMIT6PtZlDPiBzVQ/ms.jpg',
  u'is_claimed': True,
  u'is_closed': False,
  u'location': {u'address': [u'2275 Market St'],
   u'city': u'San Francisco',
   u'coordinate': {u'latitude': 37.7644154700871,
    u'longitude': -122.431834858528},
   u'country_code': u'US',
   u'cross_streets': u'Sanchez St & 16th St',
   u'display_address': [u'2275 Market St',
    u'Castro',
    u'San Francisco, CA 94114'],
   u'geo_accuracy': 9.5,
   u'neighborhoods': [u'Castro'],
   u'postal_code': u'94114',
   u'state_code': u'CA'},
  u'menu_date_updated': 1473338563,
  u'menu_provider': u'single_platform',
  u'mobile_url': u'https://m.yelp.com/biz/dinosaurs-san-francisco-4?adjust_creative=uFKYftN0fu4pD6guW2cYng&utm_campaign=yelp_api&utm_medium=api_v2_search&utm_source=uFKYftN0fu4pD

In [110]:
pd.DataFrame(data)

,categories,display_phone,id,image_url,is_claimed,is_closed,location,menu_date_updated,menu_provider,mobile_url,name,phone,rating,rating_img_url,rating_img_url_large,rating_img_url_small,review_count,snippet_image_url,snippet_text,url
0,"[[Vietnamese, vietnamese], [Sandwiches, sandwi...",+1-415-503-1421,dinosaurs-san-francisco-4,https://s3-media3.fl.yelpcdn.com/bphoto/oX333w...,True,False,"{u'cross_streets': u'Sanchez St & 16th St', u'...",1.473339e+09,single_platform,https://m.yelp.com/biz/dinosaurs-san-francisco...,Dinosaurs,4155031421,4.0,https://s3-media4.fl.yelpcdn.com/assets/2/www/...,https://s3-media2.fl.yelpcdn.com/assets/2/www/...,https://s3-media4.fl.yelpcdn.com/assets/2/www/...,744,https://s3-media2.fl.yelpcdn.com/photo/33WmpD1...,My new favorite place! Everything we tried on ...,https://www.yelp.com/biz/dinosaurs-san-francis...
1,"[[Hawaiian, hawaiian], [Asian Fusion, asianfus...",+1-628-777-6772,poke-delish-san-francisco,https://s3-media2.fl.yelpcdn.com/bphoto/G_EKLV...,True,False,"{u'cross_streets': u'Sanchez St & 15th St', u'...",NaN,NaN,https://m.yelp.com/biz/poke-delish-san-francis...,Poke Delish,6287776772,4.0,https://s3-media4.fl.yelpcdn.com/assets/2/www/...,https://s3-media2.fl.yelpcdn.com/assets/2/www/...,https://s3-media4.fl.yelpcdn.com/assets/2/www/...,139,https://s3-media4.fl.yelpcdn.com/photo/03VEsGF...,"This place was extraordinarily satisfying, and...",https://www.yelp.com/biz/poke-delish-san-franc...
2,"[[Vietnamese, vietnamese]]",+1-415-522-6689,dragoneats-san-francisco-7,https://s3-media4.fl.yelpcdn.com/bphoto/ksc8vP...,False,False,"{u'cross_streets': u'Cole St & Belvedere St', ...",NaN,NaN,https://m.yelp.com/biz/dragoneats-san-francisc...,DragonEats,4155226689,4.5,https://s3-media2.fl.yelpcdn.com/assets/2/www/...,https://s3-media4.fl.yelpcdn.com/assets/2/www/...,https://s3-media2.fl.yelpcdn.com/assets/2/www/...,68,https://s3-media2.fl.yelpcdn.com/photo/GRjmOFX...,I'm pretty cynical about SF's Bahn Mi scene af...,https://www.yelp.com/biz/dragoneats-san-franci...
3,"[[Tapas Bars, tapas], [Latin American, latin]]",+1-415-386-7670,cha-cha-cha-san-francisco-4,https://s3-media4.fl.yelpcdn.com/bphoto/POnGXZ...,True,False,"{u'cross_streets': u'Shrader St & Stanyan St',...",1.478257e+09,single_platform,https://m.yelp.com/biz/cha-cha-cha-san-francis...,Cha Cha Cha,4153867670,4.0,https://s3-media4.fl.yelpcdn.com/assets/2/www/...,https://s3-media2.fl.yelpcdn.com/assets/2/www/...,https://s3-media4.fl.yelpcdn.com/assets/2/www/...,2777,https://s3-media1.fl.yelpcdn.com/photo/ORAwhkw...,Came in late on a Friday night with some frien...,https://www.yelp.com/biz/cha-cha-cha-san-franc...
4,"[[Mediterranean, mediterranean], [Greek, greek]]",+1-415-400-5458,souvla-san-francisco-3,https://s3-media3.fl.yelpcdn.com/bphoto/TNXubv...,True,False,"{u'cross_streets': u'Fell St & Hayes St', u'ci...",NaN,NaN,https://m.yelp.com/biz/souvla-san-francisco-3?...,Souvla,4154005458,4.5,https://s3-media2.fl.yelpcdn.com/assets/2/www/...,https://s3-media4.fl.yelpcdn.com/assets/2/www/...,https://s3-media2.fl.yelpcdn.com/assets/2/www/...,107,https://s3-media4.fl.yelpcdn.com/photo/PtLkBqg...,Outstanding food. Get anything that has chicke...,https://www.yelp.com/biz/souvla-san-francisco-...
5,"[[Filipino, filipino]]",+1-415-644-8149,fob-kitchen-san-francisco-4,https://s3-media1.fl.yelpcdn.com/bphoto/yBunQu...,True,False,"{u'cross_streets': u'19th St & 20th St', u'cit...",NaN,NaN,https://m.yelp.com/biz/fob-kitchen-san-francis...,FOB Kitchen,4156448149,4.5,https://s3-media2.fl.yelpcdn.com/assets/2/www/...,https://s3-media4.fl.yelpcdn.com/assets/2/www/...,https://s3-media2.fl.yelpcdn.com/assets/2/www/...,74,https://s3-media3.fl.yelpcdn.com/photo/_9f2aPT...,Came here for a friend's bday and at first I t...,https://www.yelp.com/biz/fob-kitchen-san-franc...
6,"[[Italian, italian]]",+1-415-285-1200,barzotto-san-francisco,https://s3-media3.fl.yelpcdn.com/bphoto/RsAMyZ...,True,False,"{u'cross_streets': u'24th St & 23rd St', u'cit...",1.484904e+09,eat